# Advanced Tensor Types and Variables in TensorFlow

This notebook explores advanced tensor types and variables in TensorFlow:
- **Ragged Tensors**: For handling sequences of different lengths
- **Sparse Tensors**: For efficiently storing tensors with many zero values
- **String Tensors**: For working with text data
- **Variables**: For mutable tensors that can be updated during training

In [2]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

TensorFlow version: 2.18.1
NumPy version: 2.0.2


## 1. Ragged Tensors

Ragged tensors are tensors with non-uniform shapes along one or more dimensions. They're perfect for handling sequences of different lengths, like sentences in natural language processing or variable-length time series.

In [3]:
# Creating ragged tensors from nested Python lists
# Example: Sentences with different word counts
sentences = [
    ["I", "love", "machine", "learning"],
    ["TensorFlow", "is", "awesome"],
    ["Deep", "learning", "models", "are", "powerful", "tools"]
]

ragged_sentences = tf.ragged.constant(sentences)
print("Ragged tensor from sentences:")
print(ragged_sentences)
print(f"Shape: {ragged_sentences.shape}")
print(f"Dtype: {ragged_sentences.dtype}")

# Another example with numbers
numbers = [[1, 2, 3, 4], [5, 6], [7, 8, 9, 10, 11]]
ragged_numbers = tf.ragged.constant(numbers)
print(f"\nRagged tensor from numbers: {ragged_numbers}")
print(f"Shape: {ragged_numbers.shape}")

Ragged tensor from sentences:
<tf.RaggedTensor [[b'I', b'love', b'machine', b'learning'],
 [b'TensorFlow', b'is', b'awesome'],
 [b'Deep', b'learning', b'models', b'are', b'powerful', b'tools']]>
Shape: (3, None)
Dtype: <dtype: 'string'>

Ragged tensor from numbers: <tf.RaggedTensor [[1, 2, 3, 4], [5, 6], [7, 8, 9, 10, 11]]>
Shape: (3, None)


In [4]:
# Creating ragged tensors from value and row_splits
values = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9])
row_splits = tf.constant([0, 4, 6, 9])  # Boundaries for each row

ragged_from_splits = tf.RaggedTensor.from_row_splits(values, row_splits)
print("Ragged tensor from row splits:")
print(ragged_from_splits)

# Creating using from_row_lengths
row_lengths = tf.constant([4, 2, 3])  # Length of each row
ragged_from_lengths = tf.RaggedTensor.from_row_lengths(values, row_lengths)
print(f"\nRagged tensor from row lengths: {ragged_from_lengths}")

# Verify they're the same
print(f"\nAre they equal? {tf.reduce_all(ragged_from_splits == ragged_from_lengths)}")

Ragged tensor from row splits:
<tf.RaggedTensor [[1, 2, 3, 4], [5, 6], [7, 8, 9]]>

Ragged tensor from row lengths: <tf.RaggedTensor [[1, 2, 3, 4], [5, 6], [7, 8, 9]]>

Are they equal? True


In [5]:
# Operations on ragged tensors
print("Original ragged tensor:")
print(ragged_numbers)

# Mathematical operations
doubled = ragged_numbers * 2
print(f"\nDoubled values: {doubled}")

# Reduction operations
row_sums = tf.reduce_sum(ragged_numbers, axis=1)
print(f"\nSum of each row: {row_sums}")

total_sum = tf.reduce_sum(ragged_numbers)
print(f"Total sum: {total_sum}")

# Length operations
row_lengths = tf.map_fn(tf.size, ragged_numbers, dtype=tf.int32)
print(f"Length of each row: {row_lengths}")

# Indexing
print(f"\nFirst row: {ragged_numbers[0]}")
print(f"Element at [1, 1]: {ragged_numbers[1, 1]}")

# Converting to regular tensor (if possible)
try:
    regular_tensor = ragged_numbers.to_tensor(default_value=-1)
    print(f"\nConverted to regular tensor (padded with -1):")
    print(regular_tensor)
except Exception as e:
    print(f"Error converting to tensor: {e}")

Original ragged tensor:
<tf.RaggedTensor [[1, 2, 3, 4], [5, 6], [7, 8, 9, 10, 11]]>

Doubled values: <tf.RaggedTensor [[2, 4, 6, 8], [10, 12], [14, 16, 18, 20, 22]]>

Sum of each row: [10 11 45]
Total sum: 66
Length of each row: [4 2 5]

First row: [1 2 3 4]
Element at [1, 1]: 6

Converted to regular tensor (padded with -1):
tf.Tensor(
[[ 1  2  3  4 -1]
 [ 5  6 -1 -1 -1]
 [ 7  8  9 10 11]], shape=(3, 5), dtype=int32)


## 2. Sparse Tensors

Sparse tensors efficiently represent tensors that have many zero values. They store only the non-zero values along with their indices, which can save significant memory for large, sparse datasets.

In [6]:
# Creating sparse tensors
# Method 1: Using indices, values, and dense_shape
indices = [[0, 1], [1, 0], [2, 3], [3, 2]]  # Locations of non-zero values
values = [10, 20, 30, 40]  # Non-zero values
dense_shape = [4, 4]  # Shape of the full tensor

sparse_tensor = tf.SparseTensor(
    indices=indices,
    values=values,
    dense_shape=dense_shape
)

print("Sparse tensor:")
print(sparse_tensor)
print(f"\nIndices:\n{sparse_tensor.indices}")
print(f"\nValues:\n{sparse_tensor.values}")
print(f"\nDense shape: {sparse_tensor.dense_shape}")

# Convert to dense tensor to visualize
dense_version = tf.sparse.to_dense(sparse_tensor)
print(f"\nDense representation:\n{dense_version}")

Sparse tensor:
SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]
 [3 2]], shape=(4, 2), dtype=int64), values=tf.Tensor([10 20 30 40], shape=(4,), dtype=int32), dense_shape=tf.Tensor([4 4], shape=(2,), dtype=int64))

Indices:
[[0 1]
 [1 0]
 [2 3]
 [3 2]]

Values:
[10 20 30 40]

Dense shape: [4 4]

Dense representation:
[[ 0 10  0  0]
 [20  0  0  0]
 [ 0  0  0 30]
 [ 0  0 40  0]]


In [7]:
# Creating sparse tensor from dense tensor
dense_matrix = tf.constant([
    [0, 0, 3, 0],
    [0, 0, 0, 0],
    [0, 7, 0, 0],
    [2, 0, 0, 9]
], dtype=tf.float32)

print("Original dense matrix:")
print(dense_matrix)

# Convert dense to sparse
sparse_from_dense = tf.sparse.from_dense(dense_matrix)
print(f"\nSparse version:")
print(sparse_from_dense)

# Operations on sparse tensors
# Element-wise operations
scaled_sparse = tf.sparse.map_values(lambda x: x * 2, sparse_tensor)
print(f"\nScaled sparse tensor (values * 2):")
print(tf.sparse.to_dense(scaled_sparse))

# Sparse matrix multiplication
# Create another sparse matrix
indices2 = [[0, 0], [1, 1], [2, 2], [3, 3]]
values2 = [1, 2, 3, 4]
sparse_tensor2 = tf.SparseTensor(indices2, values2, [4, 4])

print(f"\nSecond sparse tensor (identity-like):")
print(tf.sparse.to_dense(sparse_tensor2))

# Matrix multiplication
result = tf.sparse.sparse_dense_matmul(sparse_tensor, tf.sparse.to_dense(sparse_tensor2))
print(f"\nMatrix multiplication result:")
print(result)

Original dense matrix:
tf.Tensor(
[[0. 0. 3. 0.]
 [0. 0. 0. 0.]
 [0. 7. 0. 0.]
 [2. 0. 0. 9.]], shape=(4, 4), dtype=float32)

Sparse version:
SparseTensor(indices=tf.Tensor(
[[0 2]
 [2 1]
 [3 0]
 [3 3]], shape=(4, 2), dtype=int64), values=tf.Tensor([3. 7. 2. 9.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([4 4], shape=(2,), dtype=int64))

Scaled sparse tensor (values * 2):
tf.Tensor(
[[ 0 20  0  0]
 [40  0  0  0]
 [ 0  0  0 60]
 [ 0  0 80  0]], shape=(4, 4), dtype=int32)

Second sparse tensor (identity-like):
tf.Tensor(
[[1 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 4]], shape=(4, 4), dtype=int32)

Matrix multiplication result:
tf.Tensor(
[[  0  20   0   0]
 [ 20   0   0   0]
 [  0   0   0 120]
 [  0   0 120   0]], shape=(4, 4), dtype=int32)


In [8]:
# Advanced sparse tensor operations
print("Advanced sparse tensor operations:")

# Reduction operations
sparse_sum = tf.sparse.reduce_sum(sparse_from_dense)
print(f"Sum of all elements: {sparse_sum}")

# Sum along specific axis
sparse_sum_axis0 = tf.sparse.reduce_sum(sparse_from_dense, axis=0)
print(f"Sum along axis 0: {sparse_sum_axis0}")

# Reordering sparse tensor
reordered = tf.sparse.reorder(sparse_tensor)
print(f"\nReordered sparse tensor (same values, sorted indices):")
print(reordered)

# Adding two sparse tensors
sparse_add = tf.sparse.add(sparse_tensor, sparse_tensor2)
print(f"\nAddition of two sparse tensors:")
print(tf.sparse.to_dense(sparse_add))

# Sparse tensor slicing
sliced_sparse = tf.sparse.slice(sparse_from_dense, [1, 1], [2, 2])
print(f"\nSliced sparse tensor [1:3, 1:3]:")
print(tf.sparse.to_dense(sliced_sparse))

Advanced sparse tensor operations:
Sum of all elements: 21.0
Sum along axis 0: [2. 7. 3. 9.]

Reordered sparse tensor (same values, sorted indices):
SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]
 [3 2]], shape=(4, 2), dtype=int64), values=tf.Tensor([10 20 30 40], shape=(4,), dtype=int32), dense_shape=tf.Tensor([4 4], shape=(2,), dtype=int64))

Addition of two sparse tensors:
tf.Tensor(
[[ 1 10  0  0]
 [20  2  0  0]
 [ 0  0  3 30]
 [ 0  0 40  4]], shape=(4, 4), dtype=int32)

Sliced sparse tensor [1:3, 1:3]:
tf.Tensor(
[[0. 0.]
 [7. 0.]], shape=(2, 2), dtype=float32)


## 3. String Tensors

TensorFlow provides excellent support for working with string data, including text preprocessing, encoding, and various string operations that are useful for NLP tasks.

In [9]:
# Creating string tensors
text_data = ["Hello World", "TensorFlow", "Deep Learning", "Natural Language Processing"]
string_tensor = tf.constant(text_data)

print("String tensor:")
print(string_tensor)
print(f"Shape: {string_tensor.shape}")
print(f"Dtype: {string_tensor.dtype}")

# Single string tensor
single_string = tf.constant("This is a single string tensor")
print(f"\nSingle string: {single_string}")

# Multidimensional string tensor
text_matrix = [
    ["Hello", "World"],
    ["Deep", "Learning"],
    ["Machine", "Intelligence"]
]
string_matrix = tf.constant(text_matrix)
print(f"\nString matrix:\n{string_matrix}")
print(f"Shape: {string_matrix.shape}")

String tensor:
tf.Tensor(
[b'Hello World' b'TensorFlow' b'Deep Learning'
 b'Natural Language Processing'], shape=(4,), dtype=string)
Shape: (4,)
Dtype: <dtype: 'string'>

Single string: b'This is a single string tensor'

String matrix:
[[b'Hello' b'World']
 [b'Deep' b'Learning']
 [b'Machine' b'Intelligence']]
Shape: (3, 2)


In [10]:
# String operations
print("String Operations:")

# String length
lengths = tf.strings.length(string_tensor)
print(f"String lengths: {lengths}")

# Convert to lowercase
lowercase = tf.strings.lower(string_tensor)
print(f"Lowercase: {lowercase}")

# Convert to uppercase  
uppercase = tf.strings.upper(string_tensor)
print(f"Uppercase: {uppercase}")

# String concatenation
prefix = tf.constant("Prefix_")
concatenated = tf.strings.join([prefix, string_tensor[0]])
print(f"Concatenated: {concatenated}")

# String splitting
sentence = tf.constant("This is a sample sentence for splitting")
split_words = tf.strings.split(sentence)
print(f"\nSplit sentence: {split_words}")

# Split with custom delimiter
csv_data = tf.constant("apple,banana,cherry,date")
split_csv = tf.strings.split(csv_data, sep=",")
print(f"Split CSV: {split_csv}")

# String replacement
text_with_replacements = tf.constant("Hello world! Hello TensorFlow!")
replaced = tf.strings.regex_replace(text_with_replacements, "Hello", "Hi")
print(f"Replaced text: {replaced}")

String Operations:
String lengths: [11 10 13 27]
Lowercase: [b'hello world' b'tensorflow' b'deep learning'
 b'natural language processing']
Uppercase: [b'HELLO WORLD' b'TENSORFLOW' b'DEEP LEARNING'
 b'NATURAL LANGUAGE PROCESSING']
Concatenated: b'Prefix_Hello World'

Split sentence: [b'This' b'is' b'a' b'sample' b'sentence' b'for' b'splitting']
Split CSV: [b'apple' b'banana' b'cherry' b'date']
Replaced text: b'Hi world! Hi TensorFlow!'


In [12]:
# Advanced string operations for NLP
print("Advanced String Operations for NLP:")

# Text normalization
text_samples = tf.constant([
    "  Hello World!  ",
    "TENSORFLOW IS AWESOME",
    "deep learning models"
])

# Strip whitespace and normalize case
normalized = tf.strings.strip(text_samples)
normalized = tf.strings.lower(normalized)
print(f"Normalized text: {normalized}")

# Substring operations
sample_text = tf.constant("TensorFlow Deep Learning")
substring = tf.strings.substr(sample_text, 0, 10)  # First 10 characters
print(f"Substring: {substring}")

# String to number conversion
numeric_strings = tf.constant(["123", "456.789", "999"])
string_to_number = tf.strings.to_number(numeric_strings)
print(f"String to numbers: {string_to_number}")
print(f"Converted dtype: {string_to_number.dtype}")

# Unicode operations
unicode_text = tf.constant("Héllo Wörld! 🌍")

# Get unicode length by decoding to codepoints first
unicode_codepoints = tf.strings.unicode_decode(unicode_text, "UTF-8")
unicode_length = tf.size(unicode_codepoints)

print(f"\nUnicode text: {unicode_text}")
print(f"Unicode length (character count): {unicode_length}")
print(f"Byte length: {tf.strings.length(unicode_text)}")

print(f"Unicode codepoints: {unicode_codepoints}")

# Encode back to string
encoded_back = tf.strings.unicode_encode(unicode_codepoints, "UTF-8")
print(f"Encoded back: {encoded_back}")

# Additional unicode operations
# Split unicode text into individual characters
unicode_chars = tf.strings.unicode_split(unicode_text, "UTF-8")
print(f"Unicode characters: {unicode_chars}")

# Unicode normalization (if available)
try:
    normalized_unicode = tf.strings.unicode_normalize(unicode_text, "NFD")
    print(f"Unicode normalized (NFD): {normalized_unicode}")
except AttributeError:
    print("Unicode normalization not available in this TensorFlow version")

Advanced String Operations for NLP:
Normalized text: [b'hello world!' b'tensorflow is awesome' b'deep learning models']
Substring: b'TensorFlow'
String to numbers: [123.    456.789 999.   ]
Converted dtype: <dtype: 'float32'>

Unicode text: b'H\xc3\xa9llo W\xc3\xb6rld! \xf0\x9f\x8c\x8d'
Unicode length (character count): 14
Byte length: 19
Unicode codepoints: [    72    233    108    108    111     32     87    246    114    108
    100     33     32 127757]
Encoded back: b'H\xc3\xa9llo W\xc3\xb6rld! \xf0\x9f\x8c\x8d'
Unicode characters: [b'H' b'\xc3\xa9' b'l' b'l' b'o' b' ' b'W' b'\xc3\xb6' b'r' b'l' b'd' b'!'
 b' ' b'\xf0\x9f\x8c\x8d']
Unicode normalization not available in this TensorFlow version


## 4. Variables

Variables are mutable tensors that maintain their state across function calls. They're essential for machine learning as they store model parameters (weights and biases) that need to be updated during training.

In [13]:
# Creating variables
print("Creating Variables:")

# Simple variable creation
weight = tf.Variable(5.0, name="weight")
print(f"Weight variable: {weight}")
print(f"Value: {weight.numpy()}")
print(f"Dtype: {weight.dtype}")
print(f"Shape: {weight.shape}")

# Matrix variable
weight_matrix = tf.Variable(tf.random.normal([3, 4]), name="weight_matrix")
print(f"\nWeight matrix variable:")
print(weight_matrix)

# Variable with specific initialization
bias = tf.Variable(tf.zeros([4]), name="bias")
print(f"\nBias variable (initialized with zeros): {bias}")

# Variable from existing tensor
initial_tensor = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
tensor_var = tf.Variable(initial_tensor, name="tensor_variable")
print(f"\nVariable from tensor: {tensor_var}")

# Check if it's a variable
print(f"\nIs weight a variable? {isinstance(weight, tf.Variable)}")
print(f"Is regular tensor a variable? {isinstance(tf.constant(1.0), tf.Variable)}")

Creating Variables:
Weight variable: <tf.Variable 'weight:0' shape=() dtype=float32, numpy=5.0>
Value: 5.0
Dtype: <dtype: 'float32'>
Shape: ()

Weight matrix variable:
<tf.Variable 'weight_matrix:0' shape=(3, 4) dtype=float32, numpy=
array([[-0.8210344 , -0.5605073 , -1.6817166 ,  0.20535387],
       [ 1.5647713 , -1.8431635 ,  0.04277107,  0.41162834],
       [-1.056175  , -0.3251618 ,  1.0506687 , -0.26045057]],
      dtype=float32)>

Bias variable (initialized with zeros): <tf.Variable 'bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

Variable from tensor: <tf.Variable 'tensor_variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Is weight a variable? True
Is regular tensor a variable? False


In [14]:
# Variable operations and updates
print("Variable Operations:")

print(f"Initial weight value: {weight.numpy()}")

# Update variable using assign
weight.assign(10.0)
print(f"After assign(10.0): {weight.numpy()}")

# Update using assign_add
weight.assign_add(5.0)
print(f"After assign_add(5.0): {weight.numpy()}")

# Update using assign_sub  
weight.assign_sub(3.0)
print(f"After assign_sub(3.0): {weight.numpy()}")

# Mathematical operations (creates new tensors, doesn't modify variable)
result = weight * 2
print(f"\nweight * 2 = {result.numpy()}")
print(f"Original weight unchanged: {weight.numpy()}")

# Matrix variable operations
print(f"\nOriginal weight matrix:")
print(weight_matrix.numpy())

# Update specific elements
weight_matrix[0, 0].assign(99.0)
print(f"\nAfter updating element [0,0] to 99:")
print(weight_matrix.numpy())

# Update entire rows or slices
new_row = tf.constant([1.0, 2.0, 3.0, 4.0])
weight_matrix[1, :].assign(new_row)
print(f"\nAfter updating row 1:")
print(weight_matrix.numpy())

Variable Operations:
Initial weight value: 5.0
After assign(10.0): 10.0
After assign_add(5.0): 15.0
After assign_sub(3.0): 12.0

weight * 2 = 24.0
Original weight unchanged: 12.0

Original weight matrix:
[[-0.8210344  -0.5605073  -1.6817166   0.20535387]
 [ 1.5647713  -1.8431635   0.04277107  0.41162834]
 [-1.056175   -0.3251618   1.0506687  -0.26045057]]

After updating element [0,0] to 99:
[[ 9.9000000e+01 -5.6050730e-01 -1.6817166e+00  2.0535387e-01]
 [ 1.5647713e+00 -1.8431635e+00  4.2771067e-02  4.1162834e-01]
 [-1.0561750e+00 -3.2516181e-01  1.0506687e+00 -2.6045057e-01]]

After updating row 1:
[[99.         -0.5605073  -1.6817166   0.20535387]
 [ 1.          2.          3.          4.        ]
 [-1.056175   -0.3251618   1.0506687  -0.26045057]]


In [15]:
# Variables in training simulation
print("Variables in Training Simulation:")

# Simulate a simple linear model: y = wx + b
class SimpleLinearModel:
    def __init__(self):
        # Initialize weights and bias as variables
        self.w = tf.Variable(tf.random.normal([1]), name='weight')
        self.b = tf.Variable(tf.zeros([1]), name='bias')
    
    def __call__(self, x):
        return self.w * x + self.b
    
    def get_variables(self):
        return [self.w, self.b]

# Create model
model = SimpleLinearModel()
print(f"Initial weight: {model.w.numpy()}")
print(f"Initial bias: {model.b.numpy()}")

# Generate some sample data
x_data = tf.constant([1.0, 2.0, 3.0, 4.0])
y_true = tf.constant([2.0, 4.0, 6.0, 8.0])  # True relationship: y = 2x

# Forward pass
y_pred = model(x_data)
print(f"\nPredictions: {y_pred.numpy()}")
print(f"True values: {y_true.numpy()}")

# Compute loss (mean squared error)
loss = tf.reduce_mean(tf.square(y_pred - y_true))
print(f"Initial loss: {loss.numpy()}")

# Manual gradient calculation and update (simplified)
learning_rate = 0.01
with tf.GradientTape() as tape:
    y_pred = model(x_data)
    loss = tf.reduce_mean(tf.square(y_pred - y_true))

gradients = tape.gradient(loss, model.get_variables())
print(f"\nGradients - Weight: {gradients[0].numpy()}, Bias: {gradients[1].numpy()}")

# Update variables using gradients
model.w.assign_sub(learning_rate * gradients[0])
model.b.assign_sub(learning_rate * gradients[1])

print(f"\nAfter one update step:")
print(f"Updated weight: {model.w.numpy()}")
print(f"Updated bias: {model.b.numpy()}")

# Check new loss
new_y_pred = model(x_data)
new_loss = tf.reduce_mean(tf.square(new_y_pred - y_true))
print(f"New loss: {new_loss.numpy()}")
print(f"Loss reduction: {loss.numpy() - new_loss.numpy()}")

Variables in Training Simulation:
Initial weight: [-0.6460961]
Initial bias: [0.]

Predictions: [-0.6460961 -1.2921922 -1.9382883 -2.5843844]
True values: [2. 4. 6. 8.]
Initial loss: 52.51368713378906

Gradients - Weight: [-39.691444], Bias: [-13.230481]

After one update step:
Updated weight: [-0.24918169]
Updated bias: [0.1323048]
New loss: 36.47075271606445
Loss reduction: 16.04293441772461


In [ ]:
# Advanced variable features
print("Advanced Variable Features:")

# Variable constraints
constrained_var = tf.Variable(
    tf.constant([1.0, -2.0, 3.0]), 
    constraint=lambda x: tf.clip_by_value(x, 0.0, 10.0),  # Constrain values between 0 and 10
    name="constrained_variable"
)
print(f"Constrained variable: {constrained_var.numpy()}")

# Try to set values outside the constraint
constrained_var.assign([15.0, -5.0, 7.0])  # Values will be clipped
print(f"After assigning [15.0, -5.0, 7.0] (clipped): {constrained_var.numpy()}")

# Trainable vs non-trainable variables
trainable_var = tf.Variable(1.0, trainable=True, name="trainable")
non_trainable_var = tf.Variable(1.0, trainable=False, name="non_trainable")

print(f"\nTrainable variable is trainable: {trainable_var.trainable}")
print(f"Non-trainable variable is trainable: {non_trainable_var.trainable}")

# Variable synchronization (for distributed training)
sync_var = tf.Variable(
    initial_value=1.0,
    synchronization=tf.VariableSynchronization.ON_READ,
    aggregation=tf.VariableAggregation.MEAN,
    name="synchronized_variable"
)
print(f"\nSynchronized variable: {sync_var}")

# Variable collections and tracking - improved approach
print(f"\nAll variables created so far:")
try:
    # Try to get global variables (might not work in TF 2.x)
    global_vars = tf.compat.v1.global_variables()
    if global_vars:
        for var in global_vars:
            if hasattr(var, 'name'):
                print(f"  {var.name}: {var.shape}")
    else:
        print("  Global variables list is empty or not available in TF 2.x")
except Exception as e:
    print(f"  Could not access global variables: {e}")

# Variable checkpointing simulation - improved approach
checkpoint = tf.train.Checkpoint(
    weight=weight,
    bias=bias,
    weight_matrix=weight_matrix
)

# Better way to show checkpoint variables
checkpoint_vars = {}
try:
    # Try different ways to access checkpoint information
    if hasattr(checkpoint, '_checkpoint_dependencies'):
        deps = checkpoint._checkpoint_dependencies
        if hasattr(deps, 'keys'):
            checkpoint_vars = list(deps.keys())
        else:
            checkpoint_vars = ["weight", "bias", "weight_matrix"]  # Known variables
    else:
        checkpoint_vars = ["weight", "bias", "weight_matrix"]  # Known variables
except Exception:
    checkpoint_vars = ["weight", "bias", "weight_matrix"]  # Fallback

print(f"\nCheckpoint created with variables: {checkpoint_vars}")

# Save variable state (in practice, you'd save to disk)
variable_state = {
    'weight': weight.numpy(),
    'bias': bias.numpy(), 
    'weight_matrix': weight_matrix.numpy()
}
print(f"Variable state saved: {list(variable_state.keys())}")

# Additional variable inspection
print(f"\nVariable details:")
print(f"  weight: shape={weight.shape}, dtype={weight.dtype}, trainable={weight.trainable}")
print(f"  bias: shape={bias.shape}, dtype={bias.dtype}, trainable={bias.trainable}")
print(f"  weight_matrix: shape={weight_matrix.shape}, dtype={weight_matrix.dtype}, trainable={weight_matrix.trainable}")

## Summary and Key Takeaways

### Ragged Tensors
- **Use case**: Variable-length sequences (sentences, time series)
- **Key features**: Non-uniform shapes, memory efficient for irregular data
- **Common operations**: Indexing, reduction, conversion to/from regular tensors

### Sparse Tensors
- **Use case**: Data with many zero values (sparse matrices, embeddings)
- **Key features**: Store only non-zero values and their indices
- **Common operations**: Matrix multiplication, element-wise operations, reduction

### String Tensors
- **Use case**: Text processing, NLP tasks
- **Key features**: Unicode support, extensive string manipulation functions
- **Common operations**: Splitting, joining, case conversion, regex operations

### Variables
- **Use case**: Model parameters that need to be updated during training
- **Key features**: Mutable, persistent state, gradient tracking
- **Common operations**: assign, assign_add, assign_sub, constraint enforcement

These tensor types form the foundation for advanced TensorFlow applications, especially in deep learning and machine learning workflows.